In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

import torch.utils.data as data_utils

In [4]:
# aus csv laden zu torch und float konvertieren
# features und labels separat laden
# training_data = data_utils.TensorDataset(features_train, labels_train)

In [5]:
training_data = datasets.FashionMNIST(
#training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
#test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [6]:
training_data.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [8]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


Eigenes Beispiel wie etwas trainiert werden kann

In [9]:
import numpy as np

my_array = np.ones(28*28)
my_array = my_array.reshape(1, my_array.shape[0])
my_tensor = torch.tensor(my_array).float()
# my_tensor = torch.tensor(my_array).reshape(1,-1).float()

In [10]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        #self.linear_relu_stack = nn.Sequential(
        #    nn.Linear(28*28, 512), #Dense
        #    nn.ReLU(),
        #    nn.Linear(512, 512),
        #    nn.ReLU(),
        #    nn.Linear(512, 10)
        #)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 36), #Dense
            nn.ReLU(),
            nn.Linear(36, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=36, bias=True)
    (1): ReLU()
    (2): Linear(in_features=36, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
model(my_tensor.to(device))

tensor([[-0.0233, -0.0161,  0.0695,  0.0478,  0.0203, -0.0343, -0.0437, -0.0619,
          0.0088,  0.0085]], grad_fn=<AddmmBackward0>)

In [13]:
loss_fn = nn.CrossEntropyLoss() # backward (gradienten werden gebildet partial derivatives der loss function und gradient)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # step (update) params = params - learningrate * grad

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303838  [    0/60000]
loss: 0.695534  [ 6400/60000]
loss: 0.414666  [12800/60000]
loss: 0.569396  [19200/60000]
loss: 0.502223  [25600/60000]
loss: 0.450226  [32000/60000]
loss: 0.398551  [38400/60000]
loss: 0.570637  [44800/60000]
loss: 0.534654  [51200/60000]
loss: 0.567356  [57600/60000]
Test Error: 
 Accuracy: 83.2%, Avg loss: 0.461796 

Epoch 2
-------------------------------
loss: 0.298944  [    0/60000]
loss: 0.404177  [ 6400/60000]
loss: 0.297092  [12800/60000]
loss: 0.414331  [19200/60000]
loss: 0.471028  [25600/60000]
loss: 0.357430  [32000/60000]
loss: 0.288358  [38400/60000]
loss: 0.473514  [44800/60000]
loss: 0.497024  [51200/60000]
loss: 0.524065  [57600/60000]
Test Error: 
 Accuracy: 84.6%, Avg loss: 0.423329 

Epoch 3
-------------------------------
loss: 0.248360  [    0/60000]
loss: 0.342432  [ 6400/60000]
loss: 0.271575  [12800/60000]
loss: 0.370156  [19200/60000]
loss: 0.403484  [25600/60000]
loss: 0.376916  [32000/600

In [17]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [18]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

classes = training_data.classes

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
